In [ ]:
from db import databaseconnection
import numpy as np
from ga import networkedgeneticalgorithm as nga
import matplotlib.pyplot as plt
import networkx
%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [ ]:
dbconn = databaseconnection.DatabaseConnection('db/datastore.db')

In [ ]:
dbconn.whatSessions()

In [ ]:
data = dbconn.loadSession('2018-02-18 08:56:07')

In [ ]:
dbconn.close()

### Helper Fns.

In [ ]:
#Fitness given genome
def genomeFitness(genome):
    for individual in data['individuals']:
        if np.array_equal(individual['genome'], genome):
            return individual['fitness']
    return -1

# GA Metrics

In [ ]:
#print(data['metrics'])
fitAvg = [metric['avg'] for metric in data['metrics']]
fitMin = [metric['min'] for metric in data['metrics']]
fitMax = [metric['max'] for metric in data['metrics']]
plt.plot(fitAvg, '.');
plt.plot(fitMax, '.');
plt.plot(fitMin, '.');

# Genealogy

In [ ]:
graph = networkx.DiGraph(data['genealogy']['tree'])
graph = graph.reverse()
colors = [genomeFitness(data['genealogy']['history'][i]) for i in graph]
positions = networkx.drawing.nx_agraph.graphviz_layout(graph, prog="dot")
networkx.draw(graph, positions, node_color = colors, s=40, cmap=plt.cm.Spectral)
plt.rcParams['figure.figsize'] = (40.0, 16.0)
fits = [ individual['fitness'] for individual in data['individuals']]
sm = plt.cm.ScalarMappable(cmap=plt.cm.Spectral, norm=plt.Normalize(vmin=float(min(fits)), vmax=float(max(fits))))
sm._A = []
plt.colorbar(sm)
plt.savefig('genealogy.png')
plt.show()

In [ ]:
# print(data['metrics'])
# print(len(data['genealogy']['tree']))
# print(len(data['genealogy']['history']))
# print(len(data['individuals']))
# print(data['genealogy']['tree'][1])
# print(data['genealogy']['history'][1])
#print(data['individuals'][0]['genome'])
#print(data['individuals'][0]['fitness'])
#print(genomeFitness(data['individuals'][0]['genome']))
print(len(data['individuals']))